# 1 -- INSTALL REQUIRED LIBRARIES

In [2]:
!pip install yfinance ta

# 2 -- IMPORT LIBRARIES

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator, MACD
from ta.volatility import AverageTrueRange

# 3 -- USER INPUT

In [4]:
stock_name = input("Enter Stock Name (Example: TCS.NS, RELIANCE.NS): ")

Enter Stock Name (Example: TCS.NS, RELIANCE.NS):  TCS.NS


# 4 -- FETCH STOCK DATA

In [6]:
def fetch_stock_data(stock_name, period="1y"):
    df = yf.download(stock_name, period=period, progress=False)

    if df.empty:
        raise ValueError(f"No data found for {stock_name}")

    # Ensure clean 1D columns
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        df[col] = df[col].squeeze()

    df.dropna(inplace=True)
    return df

# Example
df = fetch_stock_data(stock_name)

# 5 -- ADD TECHNICAL INDICATORS

In [8]:
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator, SMAIndicator, MACD
from ta.volatility import AverageTrueRange
import pandas as pd
import numpy as np

def add_indicators(df):

    # 🔐 FORCE TRUE 1D SERIES (THIS SOLVES EVERYTHING)
    close = pd.Series(df["Close"].values.ravel(), index=df.index)
    high  = pd.Series(df["High"].values.ravel(), index=df.index)
    low   = pd.Series(df["Low"].values.ravel(), index=df.index)
    vol   = pd.Series(df["Volume"].values.ravel(), index=df.index)

    df["Close"] = close
    df["High"] = high
    df["Low"] = low
    df["Volume"] = vol

    rows = len(df)

    # ---------- RSI ----------
    if rows >= 14:
        df["RSI"] = RSIIndicator(close=close, window=14).rsi()
    else:
        df["RSI"] = 50

    # ---------- ATR ----------
    if rows >= 14:
        df["ATR"] = AverageTrueRange(
            high=high,
            low=low,
            close=close
        ).average_true_range()
    else:
        df["ATR"] = close.pct_change().fillna(0)

    # ---------- EMA (Dynamic by stock size) ----------
    if rows >= 100:
        df["EMA_20"] = EMAIndicator(close, window=20).ema_indicator()
        df["EMA_50"] = EMAIndicator(close, window=50).ema_indicator()
        df["STOCK_TYPE"] = "LARGE"

    elif rows >= 30:
        df["EMA_10"] = EMAIndicator(close, window=10).ema_indicator()
        df["EMA_20"] = EMAIndicator(close, window=20).ema_indicator()
        df["STOCK_TYPE"] = "SMALL"

    else:
        df["SMA_5"] = SMAIndicator(close, window=5).sma_indicator()
        df["STOCK_TYPE"] = "VERY_SMALL"

    # ---------- MACD ----------
    if rows >= 26:
        macd = MACD(close)
        df["MACD"] = macd.macd()
        df["MACD_SIGNAL"] = macd.macd_signal()
    else:
        df["MACD"] = 0
        df["MACD_SIGNAL"] = 0

    df.dropna(inplace=True)
    return df

# APPLY
df = add_indicators(df)

# 6 -- STRONG TREND CHECK (KEY FOR ACCURACY)

In [10]:
# Stock type (safe)
stock_type = str(df["STOCK_TYPE"].iloc[-1])

# 🔒 GUARANTEED SCALAR VALUES (NO WARNINGS EVER)
close = df["Close"].iloc[-1].item()
rsi = df["RSI"].iloc[-1].item()
macd = df["MACD"].iloc[-1].item()
macd_signal = df["MACD_SIGNAL"].iloc[-1].item()

trend_score = 0
trend_reason = []

# ---------- LARGE STOCK LOGIC ----------
if stock_type == "LARGE":

    ema20 = df["EMA_20"].iloc[-1].item()
    ema50 = df["EMA_50"].iloc[-1].item()

    if close > ema20 and ema20 > ema50:
        trend_score += 2
        trend_reason.append("Strong EMA uptrend")

    if macd > macd_signal:
        trend_score += 1
        trend_reason.append("MACD bullish")

    if 45 <= rsi <= 70:
        trend_score += 1
        trend_reason.append("Healthy RSI")

# ---------- SMALL STOCK LOGIC ----------
elif stock_type == "SMALL":

    ema10 = df["EMA_10"].iloc[-1].item()
    ema20 = df["EMA_20"].iloc[-1].item()

    if close > ema10 and ema10 > ema20:
        trend_score += 2
        trend_reason.append("Short EMA uptrend")

    if macd > macd_signal:
        trend_score += 1
        trend_reason.append("MACD bullish")

    if 40 <= rsi <= 65:
        trend_score += 1
        trend_reason.append("RSI supportive")

# ---------- VERY SMALL / ILLIQUID ----------
else:

    recent_avg = df["Close"].rolling(5).mean().iloc[-1].item()

    if close > recent_avg:
        trend_score += 1
        trend_reason.append("Price above recent avg")

    if rsi > 45:
        trend_score += 1
        trend_reason.append("RSI > 45")

# ---------- FINAL TREND DECISION ----------
trend_strong = trend_score >= 3

print("STOCK TYPE      :", stock_type)
print("TREND SCORE     :", trend_score, "/ 4")
print("TREND STRONG?   :", "YES ✅" if trend_strong else "NO ❌")
print("TREND REASONS   :", ", ".join(trend_reason))

STOCK TYPE      : LARGE
TREND SCORE     : 1 / 4
TREND STRONG?   : NO ❌
TREND REASONS   : Healthy RSI


# 7 -- CONFIDENCE SCORE LOGIC

In [13]:
confidence = 0

# 🔒 Scalars (safe)
rsi = df["RSI"].iloc[-1].item()
macd = df["MACD"].iloc[-1].item()
macd_signal = df["MACD_SIGNAL"].iloc[-1].item()
volatility = df["Close"].pct_change().rolling(5).std().iloc[-1].item()

# ---------- BASE FROM TREND ----------
if trend_strong:
    confidence += 45
else:
    confidence += 15   # Weak trend → low base

# ---------- LARGE STOCK CONFIDENCE ----------
if stock_type == "LARGE":

    if macd > macd_signal:
        confidence += 15

    if 50 <= rsi <= 65:
        confidence += 15
    elif rsi > 70:
        confidence -= 10   # Overbought

    # Stability bonus
    if volatility < 0.02:
        confidence += 10

# ---------- SMALL STOCK CONFIDENCE ----------
elif stock_type == "SMALL":

    if macd > macd_signal:
        confidence += 10

    if 45 <= rsi <= 60:
        confidence += 10
    elif rsi > 65:
        confidence -= 15   # More dangerous in small stocks

    # Volatility penalty
    if volatility > 0.03:
        confidence -= 15

# ---------- VERY SMALL / RISKY ----------
else:

    confidence -= 10
    if volatility > 0.04:
        confidence -= 15

# ---------- HARD SAFETY CAPS ----------
if not trend_strong:
    confidence = min(confidence, 40)

confidence = max(0, min(confidence, 85))

print("CONFIDENCE SCORE :", f"{confidence}%")

CONFIDENCE SCORE : 25%


# 8 -- BUY / NO BUY DECISION

In [15]:
decision = "NO BUY"
decision_reason = []

# Thresholds
if stock_type == "LARGE":
    min_conf = 60
elif stock_type == "SMALL":
    min_conf = 65
else:
    min_conf = 100  # blocks buying risky stocks

# ---------- DECISION LOGIC ----------
if not trend_strong:
    decision_reason.append("Trend not strong")

elif confidence < min_conf:
    decision_reason.append(f"Confidence below {min_conf}%")

else:
    decision = "BUY"
    decision_reason.append("Trend + Momentum aligned")

# ---------- OVERHEAT PROTECTION ----------
if decision == "BUY" and rsi > 72:
    decision = "NO BUY"
    decision_reason.append("RSI overheated")

# ---------- FINAL OUTPUT ----------
print("\n📌 FINAL DECISION :", decision)
print("📊 Confidence     :", f"{confidence}%")
print("📈 Trend Strength :", "Strong" if trend_strong else "Weak")
print("🧠 Reason         :", ", ".join(decision_reason))


📌 FINAL DECISION : NO BUY
📊 Confidence     : 25%
📈 Trend Strength : Weak
🧠 Reason         : Trend not strong


# 9 -- TARGETS & STOP LOSS (ATR BASED)

In [21]:
# Get latest row safely
latest = df.iloc[-1]

# Extract close and ATR as float (avoid pandas warnings)
close_price = float(latest["Close"].iloc[0]) if hasattr(latest["Close"], "iloc") else float(latest["Close"])
atr = float(latest["ATR"].iloc[0]) if hasattr(latest["ATR"], "iloc") else float(latest["ATR"])

# Calculate 3 targets
target_1 = round(close_price + atr, 2)
target_2 = round(close_price + 2*atr, 2)
target_3 = round(close_price + 3*atr, 2)

# Calculate Stop Loss
stop_loss = round(close_price - atr, 2)

# Calculate % increase/decrease from current price
t1_pct = round((target_1 - close_price)/close_price * 100, 2)
t2_pct = round((target_2 - close_price)/close_price * 100, 2)
t3_pct = round((target_3 - close_price)/close_price * 100, 2)
sl_pct = round((close_price - stop_loss)/close_price * 100, 2)

# 10 -- FINANCIALS

In [24]:
print("\n📊 STOCK ANALYSIS REPORT")
print("------------------------")
print(f"Stock Name      : {stock_name}")
print(f"Current Price   : ₹{round(close_price, 2)}")
print(f"Decision        : {decision}")
print(f"Confidence      : {round(confidence, 2)}%")

print("\n🎯 TARGETS")
print(f"Target 1        : ₹{target_1}  (+{t1_pct}%)")
print(f"Target 2        : ₹{target_2}  (+{t2_pct}%)")
print(f"Target 3        : ₹{target_3}  (+{t3_pct}%)")

print(f"\n🛑 STOP LOSS    : ₹{stop_loss}  ({sl_pct}%)")


📊 STOCK ANALYSIS REPORT
------------------------
Stock Name      : TCS.NS
Current Price   : ₹3219.8
Decision        : NO BUY
Confidence      : 25%

🎯 TARGETS
Target 1        : ₹3260.3  (+1.26%)
Target 2        : ₹3300.8  (+2.52%)
Target 3        : ₹3341.3  (+3.77%)

🛑 STOP LOSS    : ₹3179.3  (1.26%)
